In [1]:
import scipy.io
import h5py
import pandas as pd
import numpy as np
import os
import cv2

In [2]:
#Read in labels
labels = pd.read_csv('labels.txt',delimiter = ',',header=None, engine='python')
names = pd.read_csv('names.txt',delimiter = '/t',header=None, engine='python')
names = np.array(names)

labels = labels.to_numpy(dtype = 'int')
labels = np.squeeze(labels)

In [3]:
##Load in Art
numart = labels.shape[0] 

#Create dummy array
artlog = np.zeros((numart,56,56,3),dtype = 'int')

#Get addresses
img_folder = '' #Address of the folder with images goes here. Make sure you write the complete address
img_folder = '/Users/erebor/Downloads/out_img'
img_filenames = os.listdir(img_folder)
art_addr = []
for ifn in img_filenames:
    art_addr.append(img_folder + '/' + ifn)


#Load in pieces one at a time
for x in range(numart):
    newart = cv2.imread(art_addr[x])
    if len(newart.shape) == 2:
        newart = np.expand_dims(newart,axis=2)
        newart = np.concatenate((newart,newart,newart),axis=2)
    artlog[x,:,:,:] = newart

In [4]:
# Shuffle

# Create Shuffle
mixer = np.arange(numart)
np.random.shuffle(mixer)

# Apply Shuffle
artlog = artlog[mixer,:,:,:]
labels = labels[mixer]

In [5]:
# Remove Anonymous/Unknown Author Art (If you wish. Would probably help performance but reduces data by 15%)
named_art = (labels <= (names.shape[0] - 2))
artlog = artlog[named_art,:,:,:]
labels = labels[named_art]
numart = labels.shape[0]

In [8]:
## Remove Art from Artists with fewer than 10 pieces
test_passers = (labels < -2)

for x in np.arange(1,names.shape[0]+1):
    artistnum = np.count_nonzero(labels == x)
    if artistnum >= 10:
        test_passers = np.logical_or(test_passers,(labels == x))

artlog = artlog[test_passers,:,:,:]
labels = labels[test_passers]
numart = labels.shape[0]
names_ = names[labels]

array([['kons\x1a\x1a-philips, margaretha elisabeth'],
       ['meester van elsloo'],
       ['philips, jan caspar'],
       ...,
       ['calraet, abraham van'],
       ['philips'],
       ['galle, theodoor']], dtype=object)

In [10]:
print("artlog",artlog.shape)
print("labels",labels.shape)
print("numart",numart)
print("names",names_.shape)


artlog (82620, 56, 56, 3)
labels (82620,)
numart 82620
names (82620, 1)


In [ ]:
## Split Data

#Declare Split Fractions
testfrac = 0.1
valfrac = 0.1

#Divide into Splits
test_train_split = round((1-(testfrac+valfrac)) * artlog.shape[0])
test_val_split = round((1-(valfrac)) * artlog.shape[0])

artlog_train = artlog[:test_train_split,:,:,:]
artlog_test = artlog[test_train_split:test_val_split,:,:,:]
artlog_val = artlog[test_val_split:,:,:,:]

labels_train = labels[:test_train_split]
labels_test = labels[test_train_split:test_val_split]
labels_val = labels[test_val_split:]

In [ ]:
## Direct Name Labels (If you want them)
namelabels_train = names[labels_train]
namelabels_test = names[labels_test]
namelabels_val = names[labels_val]